<a href="https://colab.research.google.com/github/TehJimmmyy/SentimentalAnalysis/blob/main/Hate_Speech_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1pPv9wxaUcD35Rcim2m9XHZqb8GCSCynt
!pip install pytorch-lightning==1.2.8 --quiet
!pip install transformers --quiet

import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizerFast as BertTokenizer, BertModel
import pytorch_lightning as pl

LABEL_COLUMNS="Hate_Speech"
BERT_MODEL_NAME = 'bert-base-cased'
MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 8
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1pPv9wxaUcD35Rcim2m9XHZqb8GCSCynt
From (redirected): https://drive.google.com/uc?id=1pPv9wxaUcD35Rcim2m9XHZqb8GCSCynt&confirm=t&uuid=52d42c97-c31d-4c2b-a098-b2c9361603f2
To: /content/HateSpeechModel.ckpt
100% 1.30G/1.30G [00:21<00:00, 60.2MB/s]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
class HateSpeechTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=1e-6)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

steps_per_epoch= 25960 // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5

model = HateSpeechTagger(
  n_classes=len(LABEL_COLUMNS),
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

trained_model = HateSpeechTagger.load_from_checkpoint(
  '/content/HateSpeechModel.ckpt',
  n_classes=1,strict=False
)
trained_model.eval()
trained_model.freeze()

In [7]:
#Enter comment to tag as hate speech and click run

test_comment = "Does trick2g have his youtube in the background actual degenerate"

THRESHOLD = 50  #fifty percent threshold
encoding = tokenizer.encode_plus(
  test_comment,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

_, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.flatten().numpy()


for label, prediction in zip(LABEL_COLUMNS, test_prediction):
  prediction = prediction.astype(float) * 100
  if prediction < THRESHOLD:
    print(f"Not hate speech with a probability of %.2f %%" % prediction)
    continue
  print(f"Hate Speech with a probability of %.2f %%" % prediction)

Hate Speech with a probability of 96.66 %
